In [1]:
!git clone 'https://github.com/aakashvardhan/s10-residual-ocp.git'

Cloning into 's10-residual-ocp'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 65 (delta 28), reused 48 (delta 13), pack-reused 0
Receiving objects: 100% (65/65), 16.00 KiB | 2.29 MiB/s, done.
Resolving deltas: 100% (28/28), done.


In [4]:
import sys
sys.path.append('/content/s10-residual-ocp')
sys.path.append('/content/s10-residual-ocp/models')

In [5]:
from config import get_config
config = get_config()
from main import main
import torch
torch.manual_seed(1)

ModuleNotFoundError: No module named 'models.model'